## Exercise : Extracting data from OFET-DB to perform ML

Its great that we have data stored in a standardized manner in our database but now we need to extract data from this database in the form of a table containing descriptors [X] and property [y] to perform ML. This notebook will focus on how we can do that. 

First i want you to use the backup file called 20231206_ofetdb_v2_backup11 and add it to your pgadmin as a practice database. We will avoid working with the original database.

In [ ]:
# Connect to the database


import psycopg2
import pandas as pd
import numpy as np
import plotly.express as px


#sample connection details
# pgparams = {
#     "host": "127.0.0.1",
#     "database": "ofetdb_testenv",
#     "user": "postgres",
#     "password": "password",
#     "port": "5432",
# }


# Set max number of displayed columns and rows in Jupyter Notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


pgparams = {
    "host": "",
    "database": "",
    "user": "",
    "password": "",
    "port": "",
}

def read_select_query(query):

    with psycopg2.connect(**pgparams) as conn:

        df = pd.read_sql_query(query, conn)

    return df


## Simplest Scenario :

- assume only single polymer scenarios, no blends first (wt_frac = 1)
- assume only single solvent scenarios, no multiple solvents (vol_frac = 1)
- show device substrate information 
- show film deposition information (spin, blade, etc). We will not use the parameters for now
- dont go into detail of solution treatment, substrate pretreat and post process. Just show if treatment was performed
- show hole mobility information

Follow the code blocks below and we will eventually end up with a table containing this information 

1. Prepare a dataframe containing the sample_Id, citation_type and meta information from the experimental table (This one is done as a practice for you)

In [ ]:
## Adding all the experiment information 

# SQL query to fetch the required data
query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id;
'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

# Display the resulting DataFrame
#print(result_df)

result_df

remember to keep adding to the existing query to add more information to the table. It is going to get complicated and long soon FYI. 

2. Now to this database add the solution concentration information

3. Now to this table add the solvent information but only consider devices made from single solvent vol frac = 1

4. Now to this table add polymer information only for devices made from one polymer (no blends) (wt-frac = 1)

5. Now to this table add the device substrate information 

6. Now to this table add the film deposition type only not the parameters associated with it

7. Now to this table add the solution treatment, substrate preatreat and post process informaiton 

1 if treatment is done and 0 if no treatment



8. Now to this table add the hole mobility information 

only keep devices that have an actual hole mobility value and is not Null or NAN

This is a simple version of the dataframe result_df. There are a couple more things we can now do to the dataframe to make it more machine readable. We no longer need to use sql queries. can just work with the dataframe and pandas

9. The code block below will unpack the data stored in device_substrate_parameters and store it as columns (this one is done for you)

In [ ]:
 #unpacking the information stored in device_substrate_parameters

import pandas as pd
from pandas import json_normalize



# # Extract the 'device_substrate_parameters' column and normalize it
params_df = json_normalize(result_df['device_substrate_parameters'])

# # Concatenate the original DataFrame with the new 'params_df'
result_df = pd.concat([result_df, params_df], axis=1)

# # Drop the original 'device_substrate_parameters' column
result_df = result_df.drop('device_substrate_parameters', axis=1)

# # Display the resulting DataFrame
result_df

In [ ]:

# Drop unnecessary columns that won't be used for modeling
columns_to_drop = ['film_deposition_params', 'citation_type', 'experiment_meta', 'solvent_vol_frac', 'solvent_iupac_name', 'polymer_iupac_name', 'dielectric_material_2', 'dielectric_thickness_2', 'dielectric_1_material', 'dielectric_1_thickness', 'substrate_material']
result_df = result_df.drop(columns=columns_to_drop)

# Display the resulting DataFrame
result_df


We are now going to do something called one hot encoding to convert columns with textual information into numbers (1 and 0)

In [ ]:
# List of columns to one-hot encode
columns_to_one_hot_encode = ['film_deposition_type', 'gate_material', 'dielectric_material', 'electrode_configuration']

# Perform one-hot encoding
result_df = pd.get_dummies(result_df, columns=columns_to_one_hot_encode)

# Display the resulting DataFrame with one-hot encoding
result_df


Now lets see all the columns in this database 

In [ ]:
column_names = result_df.columns
print(column_names)


### Consolidating Descriptors

We are going to consolidate some of the descriptors into one column. 


Coating :

* film_deposition_type_MGC (dip,Dip,blade, inkjet, shear, wire) - value of 1 if any of these columns are true or else 0
* film_deposition_type_spin
* film_deposition_type_drop

Gate Material :

* gate_material_n_doped Si = ('gate_material_n-doped Si', 'gate_material_Si','gate_material_p-doped Si') 

* gate_material_other = ('gate_material_Al', 'gate_material_Au', 'gate_material_PEDOT:PSS', 'gate_material_PET','gate_material_glass')


Dielectric Material :

* dielectric_material_SiO2
* dielectric_material_other = (
        'dielectric_material_6FDA-DABC',
       'dielectric_material_CYTOP', 'dielectric_material_PAN',
       'dielectric_material_PMMA', 'dielectric_material_PTrFE',
       'dielectric_material_PVP', 'dielectric_material_Shellac',
       'dielectric_material_Si3N4')




In [ ]:
result_df

In [ ]:
## replacing the pubchem_cid with solvent boiling point

# Get unique PubChem CIDs from the 'solvent_pubchem_cid' column
unique_pubchem_cids = result_df['solvent_pubchem_cid'].unique()

# Display the unique PubChem CIDs
print(unique_pubchem_cids)



In [ ]:
# Dictionary mapping PubChem CIDs to boiling points
boiling_point_dict = {
    7964: 132,
    6212: 62,
    7239: 180.1,
    6591: 146,
    7809: 138,
    13229: 238,
    13: 213,
    8030: 84,
    1140: 111,
    7501: 145,
    241: 80,
    6344: 40,
    7503: 179
}

# Add a new column "solvent_boiling_point" based on PubChem CIDs
result_df['solvent_boiling_point'] = result_df['solvent_pubchem_cid'].map(boiling_point_dict)


# Drop unnecessary columns that won't be used for modeling
columns_to_drop = ['solvent_pubchem_cid']
result_df = result_df.drop(columns=columns_to_drop)

# Display the resulting DataFrame
result_df


In [ ]:
column_names = result_df.columns
print(column_names)

In [ ]:
result_df_P3HT = result_df[result_df['polymer_common_name'] == 'P3HT']
result_df_DPP_DTT = result_df[result_df['polymer_common_name'] != 'P3HT']


### P3HT Modeling

In [ ]:
result_df_P3HT = result_df_P3HT.drop(columns='polymer_common_name')
result_df_P3HT

In [ ]:
num_rows, num_columns = result_df_P3HT.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")


### DPP-DTT Modeling

In [ ]:
result_df_DPP_DTT = result_df_DPP_DTT.drop(columns='polymer_common_name')
result_df_DPP_DTT

In [ ]:
num_rows, num_columns = result_df_DPP_DTT.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
